In [1]:
# Zadanie zaczerpnięte ze strony Kaggle, Klasyfikacja wiadomości czy jest to SPAM czy Ham
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

sms = pd.read_csv("/home/damian/misp/Projekt/spam.csv",encoding='latin-1')
sms.shape

(5572, 5)

In [2]:
# Zaprezentowanie tekstu jako dane liczbowe - przykład
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

# Wykorzystanie biblioteki scikit learn i CountVectorizer do przedstawienia tekstu jako macierzy liczb
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # stworzenie instancji

vect.fit(simple_train)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [3]:
# Sprawdzenie dopasowanego słownictwa
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [4]:
# Zamiana na document-term matrix
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [5]:
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [7]:
pd.DataFrame(simple_train_dtm.toarray(), columns = vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [8]:
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [9]:
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


In [11]:
simple_test=["please don't call me"]

In [12]:
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [13]:
pd.DataFrame(simple_test_dtm.toarray(), columns = vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


## Działania na bazie danych

In [2]:
sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
# Usuwanie nienazwanych kolumn
for col in sms.columns:
    if 'Unnamed' in col:
        del sms[col]

In [4]:
sms.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Sprawdzenie rozmiaru tabeli, 5572 wierszy oraz 2 kolumny
sms.shape

(5572, 2)

In [6]:
# Nazwanie kolumn, Lable oraz Text
sms=sms.rename(columns={'v1': 'lable', 'v2': 'text'})

In [7]:
sms.head()

,lable,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
# Zliczenie wiadomości typu Ham oraz Spam
sms.lable.value_counts()

ham     4825
spam     747
Name: lable, dtype: int64

In [10]:
# Zamiana etykiet na wartości 0/1 - ham = 0, spam = 1
sms['lable_num'] = sms.lable.map({'ham':0,'spam': 1})

In [11]:
sms.head()

,lable,text,lable_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [12]:
x = sms.text
y = sms.lable_num
print(x.shape)
print(y.shape)

(5572,)
(5572,)


In [13]:
# Podział bazy danych na część treningową przeznaczoną do nauki oraz część testową do sprawdzenia dokładności
# 80% train(4457 wierszy)  20% test(1115 wierszy) 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2 , random_state = 0)
print(x_train.shape)
print(x_test.shape)

(4457,)
(1115,)


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
# Wektoryzacja danych train
x_train_dtm = vect.fit_transform(x_train)
x_train_dtm

<4457x7612 sparse matrix of type '<class 'numpy.int64'>'
	with 58826 stored elements in Compressed Sparse Row format>

In [15]:
# Wektoryzacja danych test
x_test_dtm= vect.transform(x_test)
x_test_dtm

<1115x7612 sparse matrix of type '<class 'numpy.int64'>'
	with 13975 stored elements in Compressed Sparse Row format>

In [17]:
# Tworzenie LogisticRegression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train_dtm,y_train)

/home/damian/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Sprawdzenie dokładności modelu
from sklearn.metrics import accuracy_score

# Wykonanie testu modelu dla danych testowych x_test
y_predictions= logreg.predict(x_test_dtm)

# Porównanie wyników otrzymanych z modelu z rzeczywistymi
accuracy = accuracy_score(y_test,y_predictions)
accuracy

0.9766816143497757

In [23]:
y_pred_prob = logreg.predict_proba(x_test_dtm)[:,1]
y_pred_prob

array([0.00156076, 0.00693113, 0.00459404, ..., 0.00626734, 0.00062867,
       0.04278702])

In [28]:
# Obliczenie obszaru pod krzywą AUC na podstawie wyników z modelu
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test,y_predictions)
auc

0.9266571025937258

In [30]:
from sklearn import metrics
metrics.confusion_matrix(y_test,y_predictions)

array([[947,   2],
       [ 24, 142]])

In [31]:
# Wiadomości które są fałszywie pozytywne 
x_test[y_test<y_predictions]

1865    Call to the number which is available in appoi...
2339    Cheers for the message Zogtorius. IåÕve been s...
Name: text, dtype: object

In [32]:
# Wiadomości które są fałszywie negatywne
x_test[y_test> y_pred_class]

2078                       85233 FREE>Ringtone!Reply REAL
1892    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
3753    Bloomberg -Message center +447797706009 Why wa...
4966    You can donate å£2.50 to UNICEF's Asian Tsunam...
1457    CLAIRE here am havin borin time & am now alone...
4067    TBS/PERSOLVO. been chasing us since Sept forå£...
5381           You have 1 new message. Call 0207-083-6089
3883    A link to your picture has been sent. You can ...
1939    More people are dogging in your area now. Call...
2913    Sorry! U can not unsubscribe yet. THE MOB offe...
1216    You have 1 new voicemail. Please call 08719181...
528     You will recieve your tone within the next 24h...
3130    LookAtMe!: Thanks for your purchase of a video...
751     You have an important customer service announc...
4247    accordingly. I repeat, just text the word ok o...
5466    http//tms. widelive.com/index. wml?id=820554ad...
3979                                   ringtoneking 84484
5449    Latest